# Create dice objects to be used in simple dice game simulations

In [13]:
import numpy as np

class Die:
    '''
    Dice have a number of sides and, unless in motion, always have one face up.  
    
    Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
    '''
    
    def __init__(self,sides):
        self.sides = sides
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
    def roll(self):
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
        return self.face_up

In [14]:
die = Die(sides=6)

In [3]:
die.face_up

2

In [4]:
die.roll()

5

In [5]:
die.face_up

5

In [69]:
import numpy as np
from collections import Counter

class Dice_Game:
    '''
    This simple multiplayer dice game is the Beebe family's variation of a popular dice game, we just call it "Dice".  
    
    Instructions:
    
    Players determine who goes first, and proceed taking turns in an order (typically clockwise around a table).
    
    Each player, in turn, proceeds by first shaking all six dice, 
    and proceeding along a decision tree of subsequent rolls.
    This tree ends in either a zero score, where a shake results in no dice count for points, 
    or the player "keeps" a number of dice which count for points.
    
    The point value of dice are determined per shake, 
    and point values do not change when mixed with dice kept from previous rolls.
    
    COUNTERS:
    
    One: 100
    Five: 50
    
    THREE-OF-A-KIND:
    Ones: 1000
    Sixes: 600
    Fives: 500
    Fours: 400
    Threes: 300
    Twos: 200
    
    STRAIGHT (1,2,3,4,5,6): 2500
    
    FLUSH (ALL-OF-A-KIND): Instant Win.
    
    On the occasion where all dice rolled are "counters", the dice must all be brought back and shook again.
    The turn proceeds as usual, adding the new shake points to the previous all-counter results.
    
    
    '''
    
    def __init__(self, players, num_dice=6):
        self.player_names = ['Player_{}'.format(name + 1) for name in [j for j in range(players)]]
        self.players = [self.Player(name=player) for player in self.player_names]
        self.dice = [self.Die(sides=6) for die in [i for i in range(num_dice)]]
        self.play_to = 7500
        self.winner = False
        self.turn = 0
        
        
    def play_game(self):
        print('PLAYING GAME')
        while self.winner == False:
            for player in self.players:
                self.player_take_turn(player)
        
    def object_list_to_faces(self,die_list):
        '''
        Function takes list (potentially including lists) of die objects and returns a list (of lists) of the die faces.
        '''

        face_list = [i.face_up if type(i) == Dice_Game.Die else [j.face_up for j in i] for i in die_list]

        return face_list
    
    def shake(self,dice_to_roll,Player):
        print('SHAKING')
        shook_dice = [die.roll() for die in dice_to_roll]
        #print('I,', Player.name, 'shook the dice:', [i.face_up for i in dice_to_roll])
        print('I,', Player.name, 'shook the dice:', self.object_list_to_faces(dice_to_roll))
        return dice_to_roll


        
    def player_take_turn(self,Player):
        Player.turn += 1
        still_to_shake = self.dice.copy()
        #kept = []
        Player.turn_shakes = 0
        #print('length of self.dice', len(self.dice))
        while len(still_to_shake) > 0:
            if Player.turn_shakes == 0:
                print('lets goooo!')
                
                shake = self.shake(still_to_shake,Player)
                Player.turn_shakes += 1
                shake_faces = [i.face_up for i in shake]
                #print('shake:',shake_faces)
                counters, bring_back = self.detect_keepable(shake)
                kept, still_to_shake, nuthin = self.keep_dice(shake, counters)
                print('still to shake faces:', self.object_list_to_faces(still_to_shake))
                #kept.append(to_keep)
                Player.learning_history.extend(kept)
                print('KEPT:',self.object_list_to_faces(kept))
                score = self.kept_dice_faces_to_score(self.object_list_to_faces(kept))
                print('SCORE:', score)
                print('FIRST JUNCTION')
                print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', self.object_list_to_faces(kept),'and my total kept dice is', self.object_list_to_faces(kept))

                if bring_back:
                    print('bringing back from first junction.............')
                    still_to_shake = self.dice.copy()
                    
                stay = self.stay_or_go(score,Player)
                if stay:
                    Player.score += score
                    print('staying with score', score, ", Player's total score", Player.score)
                    break
                
            #elif kept != [] and Player.turn_shakes > 0:
            elif Player.turn_shakes >= 1:
                if nuthin == 0:
                    
                    print("keep 'em rollin'!")
                    print('kept so far:', self.object_list_to_faces(kept))

                    shake = self.shake(still_to_shake,Player)
                    Player.turn_shakes += 1
                    shake_faces = [i.face_up for i in shake]
                    print('shake:',shake_faces)
                    counters, bring_back = self.detect_keepable(shake)
                    to_keep, still_to_shake, nuthin = self.keep_dice(shake, counters)
                    #print('objects still to shake:',still_to_shake)
                    #print('still to shake faces:', [i.face_up for i in still_to_shake])
                    print('TO KEEP:', self.object_list_to_faces(to_keep))
                    kept.extend(to_keep)
                    Player.learning_history.append(kept)
                    score = self.kept_dice_faces_to_score(self.object_list_to_faces(kept))
                    print('SCORE:', score)
                    #print('KEPT:',kept)
                    print('SECOND JUNCTION')
                    print('On turn',Player.turn,'shake',Player.turn_shakes,', I,',Player.name,'kept', self.object_list_to_faces(to_keep),'and my total kept dice is', self.object_list_to_faces(kept))
                    
                    if bring_back:
                        print('bringing back from second junction.............')
                        still_to_shake = self.dice.copy()
                        
                    stay = self.stay_or_go(score,Player)
                    if stay:
                        Player.score += score
                        print('staying with score', score, ", Player's total score", Player.score)
                        break

            
                elif nuthin == 1:
                    print("you got nuthin:'",shake_faces, ". next player's turn.")
                    break
        
                print('OUTSIDE BREAK')
                if Player.score >= self.play_to:
                    self.winner = True
        
        
        
    def detect_keepable(self, shook_dice):
        
        '''
        Method which sorts through the shook dice and returns a list (of lists) of potentially keepable dice.
        
        For example, if the dice objects shook show [1,2,5,4,4,4] detect_keepable will return: [[4,4,4],1,5]
        
        '''
        print('BEGIN DETECTING KEEPABLES')
        
        BRING_EM_BACK = 0
        
        #print('shook dice:', shook_dice)
        
        dice_faces = self.object_list_to_faces(shook_dice)
        #print('dice faces', dice_faces)
        #print('DOUBLE CHECK FACES:', [i.face_up for i in shook_dice])
        
        three_of_a_kinds = [i for i in Counter(dice_faces) if Counter(dice_faces)[i] >= 3]
        #print('three_of_a_kinds:', three_of_a_kinds)
        
        keepable_three_of_a_kinds = []
        for set_of_three in three_of_a_kinds:
            print('set_of_three',set_of_three)
            toak = []
            for i in shook_dice:
                if i.face_up == set_of_three and len(toak) <= 2:
                    toak.append(i)
            keepable_three_of_a_kinds.append(toak)
            #keepable_three_of_a_kinds.append([i for i in shook_dice if i.face_up == set_of_three])
            #print('appended keepable three of a kind objects:', keepable_three_of_a_kinds)

        keepable = keepable_three_of_a_kinds
        #print('keepable objects after adding three of a kinds:', keepable)
        print('keepable dice faces after adding three of a kinds:', self.object_list_to_faces(keepable))
        
        for i in shook_dice:
            if i.face_up == 1 or i.face_up == 5:
                keepable.append(i)
        #print('keepable objects after adding ones and fives:', keepable)
        print('keepable dice faces after adding ones and fives:', self.object_list_to_faces(keepable))     

        if set(dice_faces) == set([1,2,3,4,5,6]):
            keepable.append(shook_dice)
            #print('keepable objects after adding straight:', keepable)
            print('keepable dice faces after adding straight:', self.object_list_to_faces(keepable))
            
        print('total keepable list of dice faces:', self.object_list_to_faces(keepable))
        
        if all(die in keepable for die in shook_dice):
            print('total keepable list of dice faces:', self.object_list_to_faces(keepable))
            print('ALL COUNTERS.  BRING EM BACK!  NEED FUNCTION HERE!!!!')
            BRING_EM_BACK = 1
        
        return keepable, BRING_EM_BACK
    
    
    def keep_dice(self, shook_dice, keepable_list):
        '''
        Method for choosing which dice to keep.  
        
        This will in particular need to be interfaced for agent to learn.
        
        '''
        
        print('BEGIN KEEPING DICE')
        
        nuthin = 0
        
        #print('keepable object list:', keepable_list)
        #print('keepable dice faces list:', self.object_list_to_faces(keepable_list))
        #print('len keepable_list:', len(keepable_list))
        if len(keepable_list) is 0 or len(keepable_list) is 1:
            kept_dice = keepable_list
        else:
            print('TESTING KEEPABLE LIST:', keepable_list)
            indexed_choice = np.random.choice(np.arange(len(keepable_list)))
            kept_dice = [keepable_list[indexed_choice]]
        #print('kept dice objects:', kept_dice)
        print('kept dice faces:', self.object_list_to_faces(kept_dice))

        for i in kept_dice:
            #print('object i in kept_dice:', i)
            if type(i) is list:
                print('i in kept_dice is list:', [j.face_up for j in i])
                for j in i:
                    #print('j object in list i:', j)
                    #print('j.face_up in list i:', j.face_up)
                    shook_dice.remove(j)
                    #print('shook_dice removed j object:',shook_dice)
                    print('remaining shook_dice faces after removing a', j.face_up, ':', [i.face_up for i in shook_dice])
            elif i in shook_dice:
                shook_dice.remove(i)

        dice_still_to_shake = shook_dice
        print('dice still to shake faces:', [i.face_up for i in dice_still_to_shake])
        
        if kept_dice == []:
            nuthin = 1

        return kept_dice, dice_still_to_shake, nuthin

    
    def kept_dice_faces_to_score(self,kept_dice_faces):
        
        print('SCORING')

        score = 0

        if all(isinstance(i,int) for i in kept_dice_faces):
            if set(kept_dice_faces) == set([1,2,3,4,5,6]):
                score += 2500
                print('STRAIGHT', kept_dice_faces, 'SCORES:', score)
        else:
            for i in kept_dice_faces:
                if isinstance(i,list):
                    j = i[0]
                    if j >= 2:
                        score += j*100
                        print('TOAK', i, 'SCORES:', j*100)
                    else:
                        score += 1000
                        print('TOAK of ONES', i, 'SCORES:', 1000)
                else:
                    if i == 1:
                        score += 100
                        print('SINGLE', i, 'SCORES', 100)
                    elif i == 5:
                        score += 50
                        print('SINGLE', i, 'SCORES', 50)

        return score
    
    
    
    def stay_or_go(self,potential_score,Player):
        '''
        
        Function for agent to decide whether to stay with current dice, or keep shaking.
        
        FUTURE IMPLEMENTATION: Create player method(s) for more sophisticated decision-making.
        
        '''

        stay = 0

        if potential_score >= Player.threshold:
            stay = 1

        return stay
    
    
    class Die:
        '''
        Dice have a number of sides and, unless in motion, always have one face up.  

        Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
        '''

        def __init__(self,sides):
            self.sides = sides
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

        def roll(self):
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

            return self.face_up
    
    # Players as subclass, called in Dice_Game init
    class Player:
        '''
        Each player is equipped with several attributes, mainly for statistics and for future agent-based learning.
        '''
    
        def __init__(self,name):
            self.name = name
            self.turn = 0
            self.score = 0
            self.learning_history = []
            self.turn_shakes = 0
            self.game_shakes = 0
            self.threshold = 350

            


In [70]:
game = Dice_Game(players=3)

In [71]:
game.play_to

7500

In [72]:
print([i.face_up for i in game.dice])

[4, 2, 4, 4, 4, 4]


In [73]:
game.players[0].name

'Player_1'

In [74]:
shake = game.shake(game.dice,game.players[0])

SHAKING
I, Player_1 shook the dice: [4, 5, 2, 2, 3, 3]


In [75]:
game.players[0].score

0

In [76]:
game.player_take_turn(game.players[0])

lets goooo!
SHAKING
I, Player_1 shook the dice: [4, 1, 3, 4, 1, 5]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 1, 5]
total keepable list of dice faces: [1, 1, 5]
BEGIN KEEPING DICE
TESTING KEEPABLE LIST: [<__main__.Dice_Game.Die object at 0x11b428410>, <__main__.Dice_Game.Die object at 0x11b428e10>, <__main__.Dice_Game.Die object at 0x11b428090>]
kept dice faces: [1]
dice still to shake faces: [4, 1, 3, 4, 5]
still to shake faces: [4, 1, 3, 4, 5]
KEPT: [1]
SCORING
SINGLE 1 SCORES 100
SCORE: 100
FIRST JUNCTION
On turn 1 shake 1 , I, Player_1 kept [1] and my total kept dice is [1]
keep 'em rollin'!
kept so far: [1]
SHAKING
I, Player_1 shook the dice: [1, 1, 3, 6, 4]
shake: [1, 1, 3, 6, 4]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 1]
total keepable list of dice faces: [1, 1]
BEGIN KEEPING DICE
TESTING KEEPABLE L

In [77]:
game.players[0].turn

1

In [78]:
game.players[0].score

0

In [79]:
game.play_game()

PLAYING GAME
lets goooo!
SHAKING
I, Player_1 shook the dice: [6, 1, 5, 3, 3, 5]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 5, 5]
total keepable list of dice faces: [1, 5, 5]
BEGIN KEEPING DICE
TESTING KEEPABLE LIST: [<__main__.Dice_Game.Die object at 0x11b428410>, <__main__.Dice_Game.Die object at 0x11b428150>, <__main__.Dice_Game.Die object at 0x11b428090>]
kept dice faces: [1]
dice still to shake faces: [6, 5, 3, 3, 5]
still to shake faces: [6, 5, 3, 3, 5]
KEPT: [1]
SCORING
SINGLE 1 SCORES 100
SCORE: 100
FIRST JUNCTION
On turn 2 shake 1 , I, Player_1 kept [1] and my total kept dice is [1]
keep 'em rollin'!
kept so far: [1]
SHAKING
I, Player_1 shook the dice: [3, 3, 4, 6, 5]
shake: [3, 3, 4, 6, 5]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [5]
total keepable list of dice faces: [5]
BEGIN KEEPING DICE
kept dice f

dice still to shake faces: [3, 1, 2, 4]
TO KEEP: [5]
SCORING
SINGLE 1 SCORES 100
SINGLE 5 SCORES 50
SCORE: 150
SECOND JUNCTION
On turn 4 shake 2 , I, Player_2 kept [5] and my total kept dice is [1, 5]
OUTSIDE BREAK
keep 'em rollin'!
kept so far: [1, 5]
SHAKING
I, Player_2 shook the dice: [1, 6, 6, 5]
shake: [1, 6, 6, 5]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1, 5]
total keepable list of dice faces: [1, 5]
BEGIN KEEPING DICE
TESTING KEEPABLE LIST: [<__main__.Dice_Game.Die object at 0x11b428e90>, <__main__.Dice_Game.Die object at 0x11b428e10>]
kept dice faces: [5]
dice still to shake faces: [1, 6, 6]
TO KEEP: [5]
SCORING
SINGLE 1 SCORES 100
SINGLE 5 SCORES 50
SINGLE 5 SCORES 50
SCORE: 200
SECOND JUNCTION
On turn 4 shake 3 , I, Player_2 kept [5] and my total kept dice is [1, 5, 5]
OUTSIDE BREAK
keep 'em rollin'!
kept so far: [1, 5, 5]
SHAKING
I, Player_2 shook the dice: [4, 6, 2]
shake: [4, 6, 2]
BE

dice still to shake faces: [4, 5, 5]
still to shake faces: [4, 5, 5]
KEPT: [[2, 2, 2]]
SCORING
TOAK [2, 2, 2] SCORES: 200
SCORE: 200
FIRST JUNCTION
On turn 7 shake 1 , I, Player_3 kept [[2, 2, 2]] and my total kept dice is [[2, 2, 2]]
keep 'em rollin'!
kept so far: [[2, 2, 2]]
SHAKING
I, Player_3 shook the dice: [1, 4, 4]
shake: [1, 4, 4]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: [1]
total keepable list of dice faces: [1]
BEGIN KEEPING DICE
kept dice faces: [1]
dice still to shake faces: [4, 4]
TO KEEP: [1]
SCORING
TOAK [2, 2, 2] SCORES: 200
SINGLE 1 SCORES 100
SCORE: 300
SECOND JUNCTION
On turn 7 shake 2 , I, Player_3 kept [1] and my total kept dice is [[2, 2, 2], 1]
OUTSIDE BREAK
keep 'em rollin'!
kept so far: [[2, 2, 2], 1]
SHAKING
I, Player_3 shook the dice: [5, 3]
shake: [5, 3]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones a

SCORE: 150
SECOND JUNCTION
On turn 13 shake 2 , I, Player_1 kept [5] and my total kept dice is [1, 5]
OUTSIDE BREAK
keep 'em rollin'!
kept so far: [1, 5]
SHAKING
I, Player_1 shook the dice: [6, 3, 2, 2]
shake: [6, 3, 2, 2]
BEGIN DETECTING KEEPABLES
keepable dice faces after adding three of a kinds: []
keepable dice faces after adding ones and fives: []
total keepable list of dice faces: []
BEGIN KEEPING DICE
kept dice faces: []
dice still to shake faces: [6, 3, 2, 2]
TO KEEP: []
SCORING
SINGLE 1 SCORES 100
SINGLE 5 SCORES 50
SCORE: 150
SECOND JUNCTION
On turn 13 shake 3 , I, Player_1 kept [] and my total kept dice is [1, 5]
OUTSIDE BREAK
you got nuthin:' [6, 3, 2, 2] . next player's turn.
lets goooo!
SHAKING
I, Player_2 shook the dice: [2, 1, 2, 2, 6, 5]
BEGIN DETECTING KEEPABLES
set_of_three 2
keepable dice faces after adding three of a kinds: [[2, 2, 2]]
keepable dice faces after adding ones and fives: [[2, 2, 2], 1, 5]
total keepable list of dice faces: [[2, 2, 2], 1, 5]
BEGIN KEEPI

I, Player_3 shook the dice: [3, 1, 1, 5, 6, 1]
BEGIN DETECTING KEEPABLES
set_of_three 1
keepable dice faces after adding three of a kinds: [[1, 1, 1]]
keepable dice faces after adding ones and fives: [[1, 1, 1], 1, 1, 5, 1]
total keepable list of dice faces: [[1, 1, 1], 1, 1, 5, 1]
BEGIN KEEPING DICE
TESTING KEEPABLE LIST: [[<__main__.Dice_Game.Die object at 0x11b428410>, <__main__.Dice_Game.Die object at 0x11b428150>, <__main__.Dice_Game.Die object at 0x11b428090>], <__main__.Dice_Game.Die object at 0x11b428410>, <__main__.Dice_Game.Die object at 0x11b428150>, <__main__.Dice_Game.Die object at 0x11b4287d0>, <__main__.Dice_Game.Die object at 0x11b428090>]
kept dice faces: [[1, 1, 1]]
i in kept_dice is list: [1, 1, 1]
remaining shook_dice faces after removing a 1 : [3, 1, 5, 6, 1]
remaining shook_dice faces after removing a 1 : [3, 5, 6, 1]
remaining shook_dice faces after removing a 1 : [3, 5, 6]
dice still to shake faces: [3, 5, 6]
still to shake faces: [3, 5, 6]
KEPT: [[1, 1, 1]]
SCO

In [80]:
for player in game.players:
    print(player.name, player.score)

Player_1 4500
Player_2 4200
Player_3 8150


In [48]:
list_of_lists = [[1,2,3],[4,5,6]]
np.random.choice(list_of_lists)

ValueError: a must be 1-dimensional

In [52]:
a = np.random.choice(np.arange(len(list_of_lists)))
list_of_lists[a]

[4, 5, 6]